In [ ]:
just_checking_integrity=False

In [ ]:
import gc
import pandas as pd
import scipy.sparse as sps

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
valid=sps.load_npz("./drive/MyDrive/Recommendation_system/dataset/processed_data/URM_valid_bought.npz")
bought=valid.indices

In [ ]:
dataset=None
dataset_max=None
#["UCF","ICBF","ICF","Graph","TopPop100","TopPop50","TopPop15"] # "UCF_W","Graph_W","ICF_W","UCF_WW","ICBF_WW","Graph_WW","ICF_WW", "ICBF_W"
for file in ["UCF","ICBF","Graph","ICF"]:
    print(gc.collect())
    print(f"loading {file}")
    df=pd.read_csv(f"./drive/MyDrive/Recommendation_system/dataset/candidates/traditional_recs/train/{file}.csv")
    print(len(df))
    print(f"loaded {file}")
    for col in df:
        if "Max" in col:
            df_max=df[["Session_Id",col]].drop_duplicates()
        elif "Score" in col:
            df_feats=df[["Session_Id","Item_ID",col]]
    if dataset is None:
        dataset=df_feats
        dataset_max=df_max
    else:
        dataset=dataset.merge(df_feats,on=["Session_Id","Item_ID"],how="outer")
        dataset_max=dataset_max.merge(df_max,on=["Session_Id"],how="outer")
    del df


#del dataset_max
del df_feats
del df_max

0
loading UCF
18588456
loaded UCF
0
loading ICBF
16931962
loaded ICBF
0
loading Graph
9381571
loaded Graph
0
loading ICF
20341641
loaded ICF


In [ ]:
# import ast
# submission_df=pd.read_csv("./drive/MyDrive/Recommendation_system/dataset/candidates/NN/GRU/train/candidates.csv")
# if just_checking_integrity:
#     submission_df["score"]=submission_df["score"].apply(lambda x:x.replace("-inf","0")).apply(lambda x:x.replace("+inf","0")).apply(lambda x:x.replace("inf","0"))
#     submission_df["score"]=submission_df["score"].map(ast.literal_eval)
# else:
#     submission_df["score"]=submission_df["score"].map(ast.literal_eval)
# submission_df["code"]=submission_df["code"].map(ast.literal_eval)
# submission_df["rank"]=submission_df["rank"].map(ast.literal_eval)
# submission_df = submission_df.explode(['code','score', 'rank'])
# submission_df[["session_id","code","rank"]]=submission_df[["session_id","code","rank"]].astype(int)
# submission_df.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_gru","score":"score_gru"},inplace=True)

In [ ]:
# submission_df.head()

In [ ]:
# import ast
# submission_df_tr=pd.read_csv("./drive/MyDrive/Recommendation_system/dataset/candidates/NN/Transformer/train/candidates.csv")
# if just_checking_integrity:
#     submission_df_tr["score"]=submission_df_tr["score"].apply(lambda x:x.replace("-inf","0"))
#     submission_df_tr["score"]=submission_df_tr["score"].map(ast.literal_eval)
# else:
#     submission_df_tr["score"]=submission_df_tr["score"].map(ast.literal_eval)
# submission_df_tr["code"]=submission_df_tr["code"].map(ast.literal_eval)
# submission_df_tr["rank"]=submission_df_tr["rank"].map(ast.literal_eval)
# submission_df_tr = submission_df_tr.explode(['code','score', 'rank'])
# submission_df_tr[["session_id","code","rank"]]=submission_df_tr[["session_id","code","rank"]].astype(int)
# submission_df_tr.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_transformer","score":"score_transformer"},inplace=True)

In [ ]:
import ast
submission_df_LSTM=pd.read_csv("./drive/MyDrive/Recommendation_system/dataset/candidates/NN/LSTM/train/candidates.csv")
if just_checking_integrity:
    submission_df_LSTM["score"]=submission_df_LSTM["score"].apply(lambda x:x.replace("-inf","0"))
    submission_df_LSTM["score"]=submission_df_LSTM["score"].map(ast.literal_eval)
else:
    submission_df_LSTM["score"]=submission_df_LSTM["score"].map(ast.literal_eval)
submission_df_LSTM["code"]=submission_df_LSTM["code"].map(ast.literal_eval)
submission_df_LSTM["rank"]=submission_df_LSTM["rank"].map(ast.literal_eval)
submission_df_LSTM = submission_df_LSTM.explode(['code','score', 'rank'])
submission_df_LSTM[["session_id","code","rank"]]=submission_df_LSTM[["session_id","code","rank"]].astype(int)
submission_df_LSTM.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_LSTM","score":"score_LSTM"},inplace=True)

In [ ]:
# dataset=dataset.merge(submission_df,on=["Session_Id","Item_ID"],how="outer")
# dataset=dataset.merge(submission_df_tr,on=["Session_Id","Item_ID"],how="outer")
dataset=dataset.merge(submission_df_LSTM,on=["Session_Id","Item_ID"],how="outer")

dataset=dataset.merge(dataset_max,on=["Session_Id"],how="left")


In [ ]:
if just_checking_integrity:
    dataset=dataset[:int(0.5*len(dataset))]

In [ ]:
del dataset_max
# del submission_df
# del submission_df_tr

In [ ]:
import numpy as np
sessions=np.unique(dataset["Session_Id"])
train=pd.DataFrame()
train["Session_Id"]=sessions
if just_checking_integrity:
    train["Item_ID"]=bought[:len(sessions)]
else:
    train["Item_ID"]=bought
train["target"]=1

In [ ]:
gc.collect()

792

In [ ]:
dataset=dataset.merge(train,on=["Session_Id","Item_ID"],how="left")
dataset.fillna(0,inplace=True)
gc.collect()

0

In [ ]:
dataset["target"]=dataset["target"].astype("int8")
gc.collect()

0

In [ ]:
sessions_containing_bought_among_candidates=dataset[["Session_Id","target"]].groupby("Session_Id").agg(sum).reset_index()
sessions_containing_bought_among_candidates=sessions_containing_bought_among_candidates.loc[sessions_containing_bought_among_candidates["target"]>0]

dataset=dataset[dataset['Session_Id'].isin(sessions_containing_bought_among_candidates["Session_Id"].tolist())]

In [ ]:
NHITS=len(dataset['Session_Id'].unique())
HITRATE=NHITS/81600 #approximate value
HITRATE

0.7327696078431373

In [ ]:
dataset.head(20)

,Session_Id,Item_ID,Score_UCF,Score_ICBF,Score_Graph,Score_ICF,score_LSTM,rank_LSTM,Max_Score_UCF,Max_Score_ICBF,Max_Score_Graph,Max_Score_ICF,target
209,918383,55,0.054177,0.632362,0.005355,0.018442,0.001339,47.0,594.02826,7.204127,91.79825,1.092114,0
210,918383,122,0.025767,0.000000,0.000000,0.000000,0.000000,0.0,594.02826,7.204127,91.79825,1.092114,0
211,918383,189,0.129413,0.000000,0.020602,0.278846,0.000000,0.0,594.02826,7.204127,91.79825,1.092114,0
212,918383,215,0.036236,0.000000,0.000000,0.000000,0.000000,0.0,594.02826,7.204127,91.79825,1.092114,0
213,918383,260,0.039900,0.000000,0.003595,0.012055,0.000000,0.0,594.02826,7.204127,91.79825,1.092114,0
214,918383,320,0.040230,0.000000,0.000000,0.000000,0.000000,0.0,594.02826,7.204127,91.79825,1.092114,0
215,918383,357,0.040280,0.000000,0.003286,0.012149,0.000000,0.0,594.02826,7.204127,91.79825,1.092114,0
216,918383,379,0.046153,0.637329,0.004070,0.016121,0.000850,73.0,594.02826,7.204127,91.79825,1.092114,0
217,918383,421,0.064009,0.604249,0.013832,0.319644,0.000775,80.0,594.02826,7.204127,91.79825,1.092114,0
218,918383,506,0.032975,0.493767,0.003990,0.017444,0.000000,0.0,594.02826,7.204127,91.79825,1.092114,0


In [ ]:
dataset

,Session_Id,Item_ID,Score_UCF,Score_ICBF,Score_Graph,Score_ICF,score_LSTM,rank_LSTM,Max_Score_UCF,Max_Score_ICBF,Max_Score_Graph,Max_Score_ICF,target
209,918383,55,0.054177,0.632362,0.005355,0.018442,0.001339,47.0,594.028260,7.204127,91.798250,1.092114,0
210,918383,122,0.025767,0.000000,0.000000,0.000000,0.000000,0.0,594.028260,7.204127,91.798250,1.092114,0
211,918383,189,0.129413,0.000000,0.020602,0.278846,0.000000,0.0,594.028260,7.204127,91.798250,1.092114,0
212,918383,215,0.036236,0.000000,0.000000,0.000000,0.000000,0.0,594.028260,7.204127,91.798250,1.092114,0
213,918383,260,0.039900,0.000000,0.003595,0.012055,0.000000,0.0,594.028260,7.204127,91.798250,1.092114,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
39270860,999999,2034,0.000000,0.000000,0.000000,0.000000,0.001155,93.0,111.366875,9.421845,12.826448,0.786831,0
39270861,999999,1092,0.000000,0.000000,0.000000,0.000000,0.001129,96.0,111.366875,9.421845,12.826448,0.786831,0
39270862,999999,1513,0.000000,0.000000,0.000000,0.000000,0.001127,97.0,111.366875,9.421845,12.826448,0.786831,0
39270863,999999,827,0.000000,0.000000,0.000000,0.000000,0.001115,98.0,111.366875,9.421845,12.826448,0.786831,0


In [ ]:
dataset_macro_features = pd.read_csv('./drive/MyDrive/Recommendation_system/dataset/processed_data/macro_feats_NN.csv',
                                     usecols=['session_id',
                                                'session_similarity',
                                                'session_similarity_uniques', 'session_similarity_feats', 'session_similarity_feats_uniques',
                                              'length_of_session_seconds',
                   'avg_time_spent_per_item_seconds',
                   'variance_time_spent_per_item_seconds',
                   'n_seen_items',
                   'n_unique_items',])

dataset_macro_features.rename(columns={'session_id':'Session_Id'}, inplace=True)
dataset = dataset.merge(dataset_macro_features, on='Session_Id', how='left')
del dataset_macro_features

In [ ]:
#dataset.drop('Max_Score_TopPop15', axis=1, inplace=True)

## K-Fold for All tradition models + LSTM models

In [ ]:
from sklearn.model_selection import KFold
import xgboost as xgb
import os
import time

start_time = time.time()

k = 10

score = 0
kf = KFold(n_splits=k)
index = 0
for train_index, valid_index in kf.split(train):
    train_sessions = train["Session_Id"][train_index].tolist()
    valid_sessions = train["Session_Id"][valid_index].tolist()

    train_set = dataset[dataset["Session_Id"].isin(train_sessions)]
    valid_set = dataset[dataset["Session_Id"].isin(valid_sessions)]

    train_set = train_set.sort_values("Session_Id")
    valid_set = valid_set.sort_values("Session_Id")

    qids_train = train_set.groupby("Session_Id").size().values
    qids_valid = valid_set.groupby("Session_Id").size().values

    X_train = train_set.drop(["target", "Session_Id", "Item_ID"], axis=1)
    y_train = train_set["target"]

    X_valid = valid_set.drop(["target", "Session_Id", "Item_ID"], axis=1)
    y_valid = valid_set["target"]

    model = xgb.XGBRanker(
        objective="rank:pairwise",
        n_estimators=2000,
        learning_rate=0.05,
        max_depth=6,
        reg_alpha=1e-4,
        subsample=0.8,
        min_child_weight=400
    )

    model.fit(
        X_train, y_train, group=qids_train,
        eval_set=[(X_valid, y_valid)],
        eval_group=[qids_valid],
        verbose=True,
        early_stopping_rounds=20,
        eval_metric=["map", "ndcg"]
    )

    model.save_model(f'./drive/MyDrive/Recommendation_system/rankers/xgboost/all_traditional_and_lstm/model_{index}.xgb')

    # Get maximum validation score
    score += max(model.evals_result_["validation_0"]["map"])
    index += 1

end_time = time.time()
elapsed_time = end_time - start_time


/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-map:0.22282	validation_0-ndcg:0.30271
[1]	validation_0-map:0.22913	validation_0-ndcg:0.30770
[2]	validation_0-map:0.22931	validation_0-ndcg:0.30862
[3]	validation_0-map:0.22801	validation_0-ndcg:0.30744
[4]	validation_0-map:0.22848	validation_0-ndcg:0.30799
[5]	validation_0-map:0.22793	validation_0-ndcg:0.30763
[6]	validation_0-map:0.22808	validation_0-ndcg:0.30743
[7]	validation_0-map:0.22838	validation_0-ndcg:0.30812
[8]	validation_0-map:0.22867	validation_0-ndcg:0.30817
[9]	validation_0-map:0.22798	validation_0-ndcg:0.30708
[10]	validation_0-map:0.22857	validation_0-ndcg:0.30745
[11]	validation_0-map:0.22900	validation_0-ndcg:0.30830
[12]	validation_0-map:0.22897	validation_0-ndcg:0.30851
[13]	validation_0-map:0.22853	validation_0-ndcg:0.30781
[14]	validation_0-map:0.22836	validation_0-ndcg:0.30784
[15]	validation_0-map:0.22861	validation_0-ndcg:0.30817
[16]	validation_0-map:0.22893	validation_0-ndcg:0.30820
[17]	validation_0-map:0.22848	validation_0-ndcg:0.30802
[1

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [09:41:55] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-map:0.22639	validation_0-ndcg:0.30820
[1]	validation_0-map:0.23327	validation_0-ndcg:0.31490
[2]	validation_0-map:0.23577	validation_0-ndcg:0.31748
[3]	validation_0-map:0.23755	validation_0-ndcg:0.31916
[4]	validation_0-map:0.23829	validation_0-ndcg:0.31946
[5]	validation_0-map:0.23674	validation_0-ndcg:0.31833
[6]	validation_0-map:0.23687	validation_0-ndcg:0.31842
[7]	validation_0-map:0.23795	validation_0-ndcg:0.31948
[8]	validation_0-map:0.23687	validation_0-ndcg:0.31870
[9]	validation_0-map:0.23674	validation_0-ndcg:0.31847
[10]	validation_0-map:0.23671	validation_0-ndcg:0.31854
[11]	validation_0-map:0.23751	validation_0-ndcg:0.31899
[12]	validation_0-map:0.23730	validation_0-ndcg:0.31886
[13]	validation_0-map:0.23803	validation_0-ndcg:0.31941
[14]	validation_0-map:0.23861	validation_0-ndcg:0.31959
[15]	validation_0-map:0.23860	validation_0-ndcg:0.31968
[16]	validation_0-map:0.23873	validation_0-ndcg:0.32028
[17]	validation_0-map:0.23888	validation_0-ndcg:0.32040
[1

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [09:44:31] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


...

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [10:02:31] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-map:0.22467	validation_0-ndcg:0.30555
[1]	validation_0-map:0.23818	validation_0-ndcg:0.31665
[2]	validation_0-map:0.23995	validation_0-ndcg:0.31861
[3]	validation_0-map:0.23978	validation_0-ndcg:0.31849
[4]	validation_0-map:0.23965	validation_0-ndcg:0.31844
[5]	validation_0-map:0.24131	validation_0-ndcg:0.31975
[6]	validation_0-map:0.24280	validation_0-ndcg:0.32095
[7]	validation_0-map:0.24272	validation_0-ndcg:0.32076
[8]	validation_0-map:0.24283	validation_0-ndcg:0.32166
[9]	validation_0-map:0.24224	validation_0-ndcg:0.32093
[10]	validation_0-map:0.24223	validation_0-ndcg:0.32088
[11]	validation_0-map:0.24226	validation_0-ndcg:0.32091
[12]	validation_0-map:0.24190	validation_0-ndcg:0.32064
[13]	validation_0-map:0.24242	validation_0-ndcg:0.32120
[14]	validation_0-map:0.24215	validation_0-ndcg:0.32078
[15]	validation_0-map:0.24257	validation_0-ndcg:0.32109
[16]	validation_0-map:0.24252	validation_0-ndcg:0.32124
[17]	validation_0-map:0.24307	validation_0-ndcg:0.32141
[1

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [10:08:24] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-map:0.21033	validation_0-ndcg:0.28802
[1]	validation_0-map:0.22512	validation_0-ndcg:0.30231
[2]	validation_0-map:0.22787	validation_0-ndcg:0.30458
[3]	validation_0-map:0.22895	validation_0-ndcg:0.30548
[4]	validation_0-map:0.22847	validation_0-ndcg:0.30450
[5]	validation_0-map:0.22881	validation_0-ndcg:0.30505
[6]	validation_0-map:0.22954	validation_0-ndcg:0.30629
[7]	validation_0-map:0.22890	validation_0-ndcg:0.30571
[8]	validation_0-map:0.22920	validation_0-ndcg:0.30576
[9]	validation_0-map:0.22876	validation_0-ndcg:0.30553
[10]	validation_0-map:0.22882	validation_0-ndcg:0.30573
[11]	validation_0-map:0.22904	validation_0-ndcg:0.30558
[12]	validation_0-map:0.22913	validation_0-ndcg:0.30584
[13]	validation_0-map:0.22880	validation_0-ndcg:0.30560
[14]	validation_0-map:0.22865	validation_0-ndcg:0.30541
[15]	validation_0-map:0.22873	validation_0-ndcg:0.30548
[16]	validation_0-map:0.22893	validation_0-ndcg:0.30575
[17]	validation_0-map:0.22888	validation_0-ndcg:0.30568
[1

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [10:10:23] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


In [ ]:
final_performcance=score/k
print("overall MAP@100 = ", final_performcance)

overall MAP@100 =  0.24769401042476144


In [ ]:
print("normalized score ", final_performcance*HITRATE)

normalized score  0.18150264288404638


In [ ]:
print("K-Fold execution time:", elapsed_time, "seconds")

K-Fold execution time: 1780.6489338874817 seconds
